# Ocupación de la red

## Tabla de contenido <a class= "anchor" id= "table-bullet"></a>

1. [Capacidad prestador](#first-bullet)
2. [Datos básicos prestadores](#second-bullet)
3. [Ordenado](#third-bullet)
4. [Analísis Consolidado](#fourth-bullet)

### Diccionario de Variables creadas
- Llave_producto: Es la unión entre el DNI_DISTRIBUIDOR, DNI_PRESTADOR y DSPRODUCTO.
- Llave_prestador: Es la unión entre el DNI_DISTRIBUIDOR, DNI_PRESTADOR.
- Facturado_intervención: Es la suma de las horas ejecutadas por el Producto INTERVENCION ATEL, consolidadas por Llave_prestador.
- total_producto: Es la suma de las horas ejecutadas por cada Producto, consolidadas por Llave_producto.
- total_prestador: Es la suma de las horas ejecutadas por cada Distribuidor, consolidadas por Llave_prestador.
- Porcentaje_prod: Es la fracción del Total_prestador que representa el Total_producto para cada Distribuidor.
- Suma_prestador: Es la suma de Porcentaje_prod por cada Distribuidor, agrupado por LLave_prestador.
- Cuenta_prodc: Es el número de productos habilitados por Asesor, se cuenta por Llave_prestador
- Homologación: Es la verificación de la habilitación de los productos facturados en el sistema, se cruza Llave con Llave producto.

### Librerías

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import datetime as dt
import matplotlib.pyplot as mplt
import time
import datetime
import csv
#warnings.filterwarnings(action=‘ignore’,category=DeprecationWarning)
#warnings.filterwarnings(action=‘ignore’,category=FutureWarning)

## Capacidad prestador <a class= "anchor" id= "first-bullet"></a>

[Tabla de contenido](#table-bullet)


In [4]:
#Se debe extrer e informe tareas por prestador bloque con separador pipe "|"
inicio = time.time()
ruta = 'C:/Users/dianvaja/Downloads/'
file = pd.read_csv (ruta+'TAREAS_PRESTADOR_BLOQUE.csv', sep= '|', header=0)

In [5]:
### Selección de columnas
file2= pd.DataFrame(file[['DNI_PRESTADOR','NOMBRE_PRESTADOR', 'PERFIL_TARIFA', 'DNI_DISTRIBUIDOR','NOMBRE_DISTRIBUIDOR', 
                          'CDPRODUCTO', 'DSPRODUCTO', 'TIPO_DE_RED', 'CDMUNICIPIO', 'DSMUNICIPIO',
                          'FUNCION_PRESTADOR' , 'SNVALIDADO', 'CAPACIDAD','DSTIPO_PERFIL', 'DSCLASIFICACION']])
#file2.head(50)

In [6]:
#Verificación
#filtro_asesor = file2['DNI_PRESTADOR'] == 'C1047406511'
#file_asesor= file2[filtro_asesor]
#file_asesor

In [7]:
#Se deja CGR del archivo
file2 = file2 [(file2['DNI_DISTRIBUIDOR']!= 'N811005246')]
#se filtra eliminando filas con capacidad de 1 y 0
filtro_capacidad = file2['CAPACIDAD'] >1
file2 = file2[filtro_capacidad]
file2['CAPACIDAD'].unique()

array([220., 180., 200., 140., 110., 130., 170.,  20., 160., 120.,  70.,
        90., 150., 100.,  50., 240.,  75.,  60.,  40.,  80., 128.,  55.,
       192.,  30.,  65., 165.,  35., 189.,  16.])

In [8]:
#tabla tipo de red
red = file2[(file2['SNVALIDADO']=='S')]
red = pd.value_counts(red['FUNCION_PRESTADOR'])
red1 = pd.DataFrame(red)
red1.columns=['Total prestadores']

In [9]:
#Oferta asesoría y capacitación
## Se seleccionan los servicios por clasificación
fileac = file2[(file2['DSCLASIFICACION']=='CAPACITACIÓN')| (file2['DSCLASIFICACION']=='ASESORÍA')
              |(file2['DSCLASIFICACION']=='ADMINISTRATIVA') |(file2['DSCLASIFICACION']=='OPER. ASESORIA')
              |(file2['DSCLASIFICACION']=='OPER. ADMINISTRATIVA')]
cl =fileac['DSCLASIFICACION'].unique()
#Red estrategica, especializada y comercial
fileac = fileac[(fileac['TIPO_DE_RED']=='ESTRATEGICA')| (fileac['TIPO_DE_RED']=='ESPECIALIZADA')
              |(fileac['TIPO_DE_RED']=='OPERACIONES') | (file2['DSCLASIFICACION']=='APOYO') | (file2['DSCLASIFICACION']=='COMERCIAL')]
tr= fileac['TIPO_DE_RED'].unique()
#Seleccionando prestadores validos
fileac = fileac[(fileac['SNVALIDADO']=='S')]
vl = fileac['SNVALIDADO'].unique()
#Imprimir
print('Calsifiación:', cl)
print('***********')
print('Validación: ', vl)
print('***********')
print('Tipo de red: ', tr)

Calsifiación: ['ASESORÍA' 'CAPACITACIÓN' 'ADMINISTRATIVA' 'OPER. ASESORIA']
***********
Validación:  ['S']
***********
Tipo de red:  ['ESTRATEGICA' 'ESPECIALIZADA' 'OPERACIONES']


In [10]:
#se estandariza descripción de los productos con los productos nuevos
red_e_depurada = fileac
red_e_depurada.DSPRODUCTO = red_e_depurada.DSPRODUCTO.replace({ 'PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS' : '112-PLAN DE PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS', 
                                           'ORDEN Y ASEO' : '124-GESTION DE ORDEN Y ASEO', 'MOVILIDAD SEGURA Y SOSTENIBLE' : '127-GESTION DE LA MOVILIDAD SEGURA Y SOSTENIBLE',
                                           'TRABAJOS EN ALTURA' : '128-TRABAJO SEGURO EN ALTURA', 'TRABAJOS EN ESPACIOS CONFINADOS' : '129-TRABAJO SEGURO EN ESPACIOS CONFINADOS', 
                                           'TRABAJOS EN CALIENTE' : '130-SEGURIDAD EN TRABAJOS EN CALIENTE',
                                           'CONTROL DE ENERGIAS PELIGROSAS' : '131-TRABAJO SEGURO EN RIESGO MECANICO Y ENERGIAS PELIGROSAS', 
                                           'GESTIÓN INTEGRAL  DE RIESGO QUIMICOS' : '133-GESTIÓN INTEGRAL  DE RIESGO QUIMICO', 
                                           'PVE PARA LA PREVENCIÓN DEL RIESGO BIOLOGICO' : '139-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS POR RIESGO BIOLÓGICO EN SECTOR SALUD', 
                                           'PVE PARA LA PREVENCIÓN DE LA HIPOACUSIA' : '142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA POR RUIDO', 
                                           'PVE PARA LA PREVENCIÓN DE LA NEUMOCONIOSIS' : '143-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS SOBRE LA SALUD POR ASBESTO, SÍLICE Y/O CARBÓN', 
                                           'RIESGO ELECTRICO' : '157-TRABAJO SEGURO CON RIESGO ELECTRICO', 
                                           'PVE  PARA LA PREVENCIÓN DE DESORDENES MUSCULO ESQUELETICOS' : '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)', 
                                           'GESTION ADMINISTRATIVA' : '181-GESTION ADMINISTRATIVA', 'ESTRUCTURA Y GESTIÓN DEL RIESGO PSICOSOCIAL Y LA SALUD MENTAL' : '167-GESTIÓN PSICOSOCIAL Y PROMOCION DE LA SALUD MENTAL', 
                                           'SISTEMA DE GESTION EN SST' : '170-SISTEMA DE GESTIÓN EN SST', 'PVE PARA LA PREVENCION DEL RIESGO DE RADIACIONES IONIZANTES' : '172-PVE Y ACCIONES PARA PREVENCION DE LOS EFECTOS SOBRE LAS SALUD POR LAS  RADIACIONES IONIZANTES', 
                                           'PVE PARA LA PREVENCION DE RIESGO QUIMICO' : '173-PVE Y ACCIONES PARA PREVENCION DE LOS EFECTOS SOBRE LA SALUD POR AGENTES DE RIESGO QUIMICO',
                                           'APOYO A LA PROMOCIÓN Y PREVENCIÓN' : '175-APOYO A LA PROMOCIÓN Y PREVENCIÓN', 'ARIADNA' : '176-ARIADNA', 'CONTRATISTAS' : '177-CONTRATISTAS', 
                                           'CULTURA EN SEGURIDAD Y SALUD EN EL TRABAJO' : '178-CULTURA EN SEGURIDAD Y SALUD EN EL TRABAJO', 'LABORATORIO DE HIGIENE' : '182-LABORATORIO DE HIGIENE', 
                                           'LÍDERES EN SALUD Y SEGURIDAD EN EL TRABAJO' : '183-LÍDERES EN SALUD Y SEGURIDAD EN EL TRABAJO', 'MERCADEO COMO APOYO A LA PROMOCIÓN Y PREVENCIÓN' : '185-MERCADEO COMO APOYO A LA PROMOCIÓN Y PREVENCIÓN', 
                                           'PROGRAMA PARA LA PROMOCION DE LA SALUD CARDIOVASCULAR' : '186-PROGRAMA DE ESTILOS DE VIDA SALUDABLES Y PROMOCIÓN DE LA SALUD', 'PVE PARA LA PREVENCIÓN DEL RIESGO BIOLOGICO - DECRETO 488' : '188-PVE PARA LA PREVENCIÓN DEL RIESGO BIOLOGICO - DECRETO 488', 
                                           'REINCORPORACIÓN LABORAL' : '190-REINCORPORACIÓN LABORAL', 'SEGURIDAD DE PROCESOS' : '191-SEGURIDAD DE PROCESOS', 'SVE PARA RIESGO BIOLOGICO- COVID 19' : '192-SVE PARA RIESGO BIOLOGICO- COVID 19', 
                                           'SOBREESFUERZOS (ATEL)' : '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)', 'ATENCIÓN PYMES PROTOCOLO 1' : '189-PYMES', 
                                           'ATENCIÓN PYMES PROTOCOLO 2' : '189-PYMES', 'ATENCIÓN PYMES PROTOCOLO 3' : '189-PYMES', 'ATENCIÓN PYMES PROTOCOLO 4' : '189-PYMES', 
                                           'ATENCIÓN PYMES PROTOCOLO 5 (SOLO BOGOTA)' : '189-PYMES', 'PYMES' : '189-PYMES', '158-PVE Y ACCIONES PARA LA GESTIÓN DEL RIESGO BIOMECANICO':'158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)'})

c = red_e_depurada['DSPRODUCTO'].unique()
print('Productos:',c)

Productos: ['170-SISTEMA DE GESTIÓN EN SST' 'NO USAR - SOBREESFUERZOS (ATEL)'
 '124-GESTION DE ORDEN Y ASEO'
 '127-GESTION DE LA MOVILIDAD SEGURA Y SOSTENIBLE'
 'NO USAR - PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS'
 '112-PLAN DE PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS'
 'NO USAR - INTERVENCIÓN DEL  AT Y EL'
 'NO USAR - ESTRUCTURA Y GESTIÓN DEL RIESGO PSICOSOCIAL Y LA SALUD MENTAL'
 '167-GESTIÓN PSICOSOCIAL Y PROMOCION DE LA SALUD MENTAL'
 'NO USAR - SISTEMA DE GESTION EN SST'
 '139-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS POR RIESGO BIOLÓGICO EN SECTOR SALUD'
 '171-PREVENCIÓN DE CAÍDAS A NIVEL'
 '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)'
 'NO USAR - ORDEN Y ASEO'
 '142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA POR RUIDO'
 '183-LÍDERES EN SALUD Y SEGURIDAD EN EL TRABAJO'
 'NO USAR - PVE  PARA LA PREVENCIÓN DE DESORDENES MUSCULO ESQUELETICOS'
 'NO USAR - PVE

In [11]:
#se estandariza codigos de los productos con los productos nuevos
red_e_depurada.CDPRODUCTO = red_e_depurada.CDPRODUCTO.replace({12 : 112, 24 : 124, 27 : 127, 28 : 128, 29 : 129, 
                                                               30 : 130, 31 : 131, 33 : 133, 39 : 139, 42 : 142,
                                                               43 : 143, 57 : 157, 58 : 158, 61 : 181, 67 : 167, 
                                                               11 : 170, 72 : 172, 73 : 173, 21 : 175, 2 : 176, 
                                                               81 : 177, 60 : 178, 7 : 182, 23 : 183, 74 : 185, 
                                                               71 : 186, 84 : 188, 59 : 190, 70 : 191, 83 : 192, 68 : 158, 
                                                               75 : 189,76 : 189,77 : 189,78 : 189,79 : 189,
                                                               174 : 189, 

})
red_e_depurada['CDPRODUCTO'].unique()

array([170, 158, 124, 127, 112,   4, 167, 139, 171, 142, 183, 192, 197,
        13, 185, 143, 190, 172, 173, 189, 128, 131, 129, 196, 178, 133,
       130, 186, 194, 182, 181, 157, 195, 191, 177, 188, 175,  65, 176],
      dtype=int64)

In [12]:
#Quitar producto intervención del ATEL
red_e_depurada = red_e_depurada[red_e_depurada['DSPRODUCTO'] != 'INTERVENCIÓN DEL  AT Y EL']
red_e_depurada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1359953 entries, 0 to 2517470
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   DNI_PRESTADOR        1359953 non-null  object 
 1   NOMBRE_PRESTADOR     1359953 non-null  object 
 2   PERFIL_TARIFA        1359953 non-null  object 
 3   DNI_DISTRIBUIDOR     1359953 non-null  object 
 4   NOMBRE_DISTRIBUIDOR  1359858 non-null  object 
 5   CDPRODUCTO           1359953 non-null  int64  
 6   DSPRODUCTO           1359953 non-null  object 
 7   TIPO_DE_RED          1359953 non-null  object 
 8   CDMUNICIPIO          1359953 non-null  float64
 9   DSMUNICIPIO          1359953 non-null  object 
 10  FUNCION_PRESTADOR    1359745 non-null  object 
 11  SNVALIDADO           1359953 non-null  object 
 12  CAPACIDAD            1359953 non-null  float64
 13  DSTIPO_PERFIL        1359953 non-null  object 
 14  DSCLASIFICACION      1359953 non-null  object 
dty

In [13]:
#Hacer llave para eliminar duplicados
red_e_depurada['Llave'] = red_e_depurada.DNI_DISTRIBUIDOR + red_e_depurada.DNI_PRESTADOR + red_e_depurada.DSPRODUCTO
#Llave prestador
red_e_depurada['Llave_prestador'] = red_e_depurada.DNI_DISTRIBUIDOR + red_e_depurada.DNI_PRESTADOR
red_e_depurada.info()
#Eliminar duplicados
red_e_depurada = red_e_depurada.drop_duplicates(['Llave_prestador'], keep = 'last')
d = red_e_depurada.duplicated().sum()
print(d)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1359953 entries, 0 to 2517470
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   DNI_PRESTADOR        1359953 non-null  object 
 1   NOMBRE_PRESTADOR     1359953 non-null  object 
 2   PERFIL_TARIFA        1359953 non-null  object 
 3   DNI_DISTRIBUIDOR     1359953 non-null  object 
 4   NOMBRE_DISTRIBUIDOR  1359858 non-null  object 
 5   CDPRODUCTO           1359953 non-null  int64  
 6   DSPRODUCTO           1359953 non-null  object 
 7   TIPO_DE_RED          1359953 non-null  object 
 8   CDMUNICIPIO          1359953 non-null  float64
 9   DSMUNICIPIO          1359953 non-null  object 
 10  FUNCION_PRESTADOR    1359745 non-null  object 
 11  SNVALIDADO           1359953 non-null  object 
 12  CAPACIDAD            1359953 non-null  float64
 13  DSTIPO_PERFIL        1359953 non-null  object 
 14  DSCLASIFICACION      1359953 non-null  object 
 15

In [14]:
#Limpieza de datos
na = red_e_depurada.isnull().any()
na_=red_e_depurada.isnull().sum()
print(na)
print('***************************')
print(na_)
#No tiene datos nulos

DNI_PRESTADOR          False
NOMBRE_PRESTADOR       False
PERFIL_TARIFA          False
DNI_DISTRIBUIDOR       False
NOMBRE_DISTRIBUIDOR     True
CDPRODUCTO             False
DSPRODUCTO             False
TIPO_DE_RED            False
CDMUNICIPIO            False
DSMUNICIPIO            False
FUNCION_PRESTADOR       True
SNVALIDADO             False
CAPACIDAD              False
DSTIPO_PERFIL          False
DSCLASIFICACION        False
Llave                  False
Llave_prestador        False
dtype: bool
***************************
DNI_PRESTADOR          0
NOMBRE_PRESTADOR       0
PERFIL_TARIFA          0
DNI_DISTRIBUIDOR       0
NOMBRE_DISTRIBUIDOR    3
CDPRODUCTO             0
DSPRODUCTO             0
TIPO_DE_RED            0
CDMUNICIPIO            0
DSMUNICIPIO            0
FUNCION_PRESTADOR      1
SNVALIDADO             0
CAPACIDAD              0
DSTIPO_PERFIL          0
DSCLASIFICACION        0
Llave                  0
Llave_prestador        0
dtype: int64


In [15]:
#Ver nulos
nulos_prestador = red_e_depurada[red_e_depurada.FUNCION_PRESTADOR.isnull()]
nulos_prestador.head(5)

,DNI_PRESTADOR,NOMBRE_PRESTADOR,PERFIL_TARIFA,DNI_DISTRIBUIDOR,NOMBRE_DISTRIBUIDOR,CDPRODUCTO,DSPRODUCTO,TIPO_DE_RED,CDMUNICIPIO,DSMUNICIPIO,FUNCION_PRESTADOR,SNVALIDADO,CAPACIDAD,DSTIPO_PERFIL,DSCLASIFICACION,Llave,Llave_prestador
1387774,C1052380824,CABRERA CASTAÑO HERNEY,A,N900128699,IRS FORENSE LTDA - BOGOTA,127,NO USAR - MOVILIDAD SEGURA Y SOSTENIBLE,ESTRATEGICA,586.0,BOGOTA,NaN,S,180.0,AVANZADO,CAPACITACIÓN,N900128699C1052380824NO USAR - MOVILIDAD SEGUR...,N900128699C1052380824


## Datos básicos prestadores<a class= "anchor" id= "second-bullet"></a>

[Tabla de contenido](#table-bullet)

In [16]:
#Se estrae de data lake informe DATOS BASICOS PRESTADORES separado con pipe desde diciembre de 2022 a diciembre de 2023
file_basico = pd.read_csv (ruta+'DATOS BASICOS PRESTADORES.csv', sep= '|', header=0)
file_basico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75923 entries, 0 to 75922
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CDDELEGACION                75923 non-null  int64  
 1   DNI_DISTRIBUIDOR            75923 non-null  object 
 2   NOMBRE_DISTRIBUIDOR         75923 non-null  object 
 3   DNI_PRESTADOR               75923 non-null  object 
 4   NOMBRE_PRESTADOR            75923 non-null  object 
 5   TELEFONO_PPAL               72372 non-null  object 
 6   FNACIM                      71087 non-null  object 
 7   DOMICILIO                   73660 non-null  object 
 8   CDMUNICIPIO                 75923 non-null  int64  
 9   TELEFONOTRAB                40790 non-null  float64
 10  FAX                         9027 non-null   float64
 11  DSEMAIL                     64146 non-null  object 
 12  SEXO                        72063 non-null  object 
 13  POBLACION                   759

In [17]:
file_basico2= pd.DataFrame(file_basico[['DNI_PRESTADOR', 'DNI_DISTRIBUIDOR','CAPACIDAD1', 'CAPACIDAD2', 'CAPACIDAD3', 'CAPACIDAD4', 
                            'CAPACIDAD5', 'CAPACIDAD6', 'CAPACIDAD7', 'CAPACIDAD8', 'CAPACIDAD9', 'CAPACIDAD10', 'CAPACIDAD11',
                            'CAPACIDAD12']])
#file_basico2.head(50)

In [18]:
#Se filtra quita CGR
basicos_cgr = file_basico2 [(file_basico2['DNI_DISTRIBUIDOR']!= 'N811005246')]
#Se crea a llave prestador
basicos_cgr['Llave_prestador'] = basicos_cgr.DNI_DISTRIBUIDOR + basicos_cgr.DNI_PRESTADOR
#Ordenar
basicos_cgr.sort_values('CAPACIDAD6', inplace = True)
#Eliminar duplicados
basicos_cgr = basicos_cgr.drop_duplicates(['Llave_prestador'], keep = 'last')
d = basicos_cgr.duplicated().sum()
print(d)
#49812 duplicados

0


In [19]:
#Capacidad total
basicos_cgr['Capacidad_total'] = basicos_cgr['CAPACIDAD1'] + basicos_cgr['CAPACIDAD2'] + basicos_cgr['CAPACIDAD3'] + basicos_cgr['CAPACIDAD4'] + basicos_cgr['CAPACIDAD5'] + basicos_cgr['CAPACIDAD6'] + basicos_cgr['CAPACIDAD7'] + basicos_cgr['CAPACIDAD8'] + basicos_cgr['CAPACIDAD9'] + basicos_cgr['CAPACIDAD10'] + basicos_cgr['CAPACIDAD11'] + basicos_cgr['CAPACIDAD12']

In [20]:
#Unir los archivo
red_e_depurada= pd.merge (left= red_e_depurada, right= basicos_cgr, left_on= 'Llave_prestador', right_on= 'Llave_prestador', how= 'left')

In [21]:
#red_e_depurada

In [22]:
#exportar
#red_e_depurada.to_csv('Red_externa_asesoría_y_capacitación.csv', index = False)
red_e_depurada.to_excel('CAPACIDAD_MENSUAL_RED_EXTERNA.xlsx', index = False)
#file_prueba.to_excel('PRUEBA.xlsx')

## Ordenado <a class= "anchor" id= "third-bullet"></a>

[Tabla de contenido](#table-bullet)

In [23]:
#Se debe extrer el informe del ordenado de datalake pipe "|"
f = pd.read_csv (ruta+'Ordenado_tera.txt.txt', sep= '|', header=0, on_bad_lines='skip', encoding="iso-8859-1", 
                 index_col=False, quoting=csv.QUOTE_NONE)
#f = f.reset_index(drop=True)
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100677 entries, 0 to 1100676
Data columns (total 90 columns):
 #   Column                                                                                                 Non-Null Count    Dtype  
---  ------                                                                                                 --------------    -----  
 0   Ord_Plan_Vers_Act_Id                                                                                   1100677 non-null  object 
 1   Numero_Consecutivo_Orden                                                                               1100677 non-null  int64  
 2   Numero_Consecutivo_Plan                                                                                1100677 non-null  int64  
 3   Dni_Prestador_Externo                                                                                  1100677 non-null  object 
 4   Nombre_Prestador_Externo                                                      

In [24]:
file = pd.DataFrame(f[['Numero_Consecutivo_Orden', 'Numero_Consecutivo_Plan', 'Numero_Actividad', 'Numero_Version','Dni_Prestador_Externo', 'Nombre_Prestador_Externo', 'Dni_Prestador','Nombre_Prestador', 'Tipo_Asesor', 
                          'Contrato_Arl_Id','Codigo_Tarea', 'Tarea_Desc', 'Codigo_Producto', 'Producto_Desc', 'Fecha_Entrega_Servicio', 'Numero_Cantidad_Pedida', 'Numero_Saldo_Ejecutado',
                          'Estado_Orden_Desc', 'Clasificacion_Desc']]).rename(columns={'Numero_Consecutivo_Orden': 'NMCONSECUTIVO_ORDEN',
                                                                                      'Numero_Consecutivo_Plan': 'NMCONSECUTIVO_PLAN',
                                                                                      'Numero_Actividad': 'NMACTIVIDAD',
                                                                                      'Numero_Version': 'NMVERSION',
                                                                                      'Dni_Prestador_Externo': 'DNI_DISTRIBUIDOR',
                                                                                      'Nombre_Prestador_Externo': 'NOMBRE_DISTRIBUIDOR',
                                                                                      'Dni_Prestador': 'DNI_PRESTADOR',
                                                                                      'Nombre_Prestador': 'NOMBRE_PRESTADOR',
                                                                                      'Tipo_Asesor': 'TIPO_DE_ASESOR',
                                                                                      'Contrato_Arl_Id': 'NPOLIZA',
                                                                                      'Codigo_Tarea': 'CDTAREA',
                                                                                      'Tarea_Desc': 'DSTAREA',
                                                                                       'Clasificacion_Desc':'CLASIFICACION',
                                                                                      'Codigo_Producto':'CDPRODUCTO', 
                                                                                      'Producto_Desc': 'DSPRODUCTO',
                                                                                      'Fecha_Entrega_Servicio': 'FEENTREGA_SERVICIO_INI',
                                                                                      'Numero_Cantidad_Pedida': 'NMCANTIDAD_PEDIDA',
                                                                                      'Numero_Saldo_Ejecutado': 'NMCANTIDAD_EJECUTADA',
                                                                                      'Estado_Orden_Desc':'DSESTADO_PROGRAMACION'})

file.head()

,NMCONSECUTIVO_ORDEN,NMCONSECUTIVO_PLAN,NMACTIVIDAD,NMVERSION,DNI_DISTRIBUIDOR,NOMBRE_DISTRIBUIDOR,DNI_PRESTADOR,NOMBRE_PRESTADOR,TIPO_DE_ASESOR,NPOLIZA,CDTAREA,DSTAREA,CDPRODUCTO,DSPRODUCTO,FEENTREGA_SERVICIO_INI,NMCANTIDAD_PEDIDA,NMCANTIDAD_EJECUTADA,DSESTADO_PROGRAMACION,CLASIFICACION
0,5832325,93331,23,2,N811005246,CONSULTORIA EN GESTION DE RIESGOS SURAMERICANA...,C1022094940,HERNANDEZ TANGARIFE LIZETH JOHANA,ASESOR EN PREVENCIÓN,94172384,207,PREVENCION DE DESORDENES MUSCULOESQUELETICO,58,NO USAR - PVE PARA LA PREVENCIÓN DE DESORDENE...,13/05/2022 00:00:00,2.0,0.0,FACTURADO,CAPACITACIÓN
1,6241019,56801,70,11,N901204008,NOVA INGENIERÍA Y CONSULTORÍA S.A.S,C53013410,CORREA MORENO CATALINA,ASESOR EN PREVENCIÓN,94116184,2443,"NORMAS, PROCEDIMIENTOS Y CONTROLES PARA EL MAN...",133,133-GESTIÓN INTEGRAL DE RIESGO QUIMICO,01/04/2023 00:00:00,4.0,0.0,APROBADO,ASESORÍA
2,6072288,290850,38,6,N811005246,CONSULTORIA EN GESTION DE RIESGOS SURAMERICANA...,C52833584,RUIZ ASTROS INGRID JOHANA,ASESOR EN PREVENCIÓN,94371391,1932,PROGRAMA O PROCEDIMIENTO PARA LA GESTION DEL R...,27,NO USAR - MOVILIDAD SEGURA Y SOSTENIBLE,30/09/2022 00:00:00,3.0,0.0,FACTURADO,ASESORÍA
3,5422231,13353,13,13,C38568402,ESCOBAR MATERON CAROLINA,C38568402,ESCOBAR MATERON CAROLINA,ASESOR EN PREVENCIÓN,94011286,52,PRACTICAS SEGURAS,58,NO USAR - PVE PARA LA PREVENCIÓN DE DESORDENE...,01/05/2022 00:00:00,2.0,2.0,FACTURADO,CAPACITACIÓN
4,5855076,1274592,117,1,N900592369,QUIRONPREVENCION COLOMBIA S.A.S.,C1022373655,BELTRAN LOPEZ PAOLA ANDREA,?,96470682,1818,SEGUIMIENTO A INDICADORES DE GESTION E IMPACTO,4,NO USAR - INTERVENCIÓN DEL AT Y EL,25/05/2022 00:00:00,1.0,1.0,FACTURADO,ASESORÍA


In [25]:
#Se quita CGR del archivo
file = file [(file['DNI_DISTRIBUIDOR']!= 'N811005246')]
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 589566 entries, 1 to 1100676
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   NMCONSECUTIVO_ORDEN     589566 non-null  int64  
 1   NMCONSECUTIVO_PLAN      589566 non-null  int64  
 2   NMACTIVIDAD             589566 non-null  int64  
 3   NMVERSION               589566 non-null  int64  
 4   DNI_DISTRIBUIDOR        589566 non-null  object 
 5   NOMBRE_DISTRIBUIDOR     589566 non-null  object 
 6   DNI_PRESTADOR           589566 non-null  object 
 7   NOMBRE_PRESTADOR        589566 non-null  object 
 8   TIPO_DE_ASESOR          589566 non-null  object 
 9   NPOLIZA                 589566 non-null  int64  
 10  CDTAREA                 589566 non-null  int64  
 11  DSTAREA                 589566 non-null  object 
 12  CDPRODUCTO              589566 non-null  int64  
 13  DSPRODUCTO              589566 non-null  object 
 14  FEENTREGA_SERVICIO_

In [26]:
#Verificación
#filtro = f['Clasificacion_Desc'] == 3146671700
#file_prueba= f[filtro]
#file_prueba

In [27]:
#Se seleccionan los servicios a dimensionar
file = file[(file['CLASIFICACION']=='CAPACITACIÓN')| (file['CLASIFICACION']=='ASESORÍA')
              |(file['CLASIFICACION']=='ADMINISTRATIVA') |(file['CLASIFICACION']=='OPER. ASESORIA')
              |(file['CLASIFICACION']=='OPER. ADMINISTRATIVA')]
cl =file['CLASIFICACION'].unique()
#Red estrategica]
#Imprimir
print('CLASIFICACION:', cl)

CLASIFICACION: ['ASESORÍA' 'CAPACITACIÓN' 'ADMINISTRATIVA']


In [28]:
#se estandariza descripción de los productos con los productos nuevos
file.DSPRODUCTO = file.DSPRODUCTO.replace({   'NO USAR - PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS' : '112-PLAN DE PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS',
 'NO USAR - ORDEN Y ASEO' : '124-GESTION DE ORDEN Y ASEO',
 'NO USAR - MOVILIDAD SEGURA Y SOSTENIBLE' : '127-GESTION DE LA MOVILIDAD SEGURA Y SOSTENIBLE',
 'NO USAR - TRABAJOS EN ALTURA' : '128-TRABAJO SEGURO EN ALTURA',
 'NO USAR - TRABAJOS EN ESPACIOS CONFINADOS' : '129-TRABAJO SEGURO EN ESPACIOS CONFINADOS',
 'NO USAR - TRABAJOS EN CALIENTE' : '130-SEGURIDAD EN TRABAJOS EN CALIENTE',
 'NO USAR - CONTROL DE ENERGIAS PELIGROSAS' : '131-TRABAJO SEGURO EN RIESGO MECANICO Y ENERGIAS PELIGROSAS',
 'NO USAR - GESTIÓN INTEGRAL  DE RIESGO QUIMICOS' : '133-GESTIÓN INTEGRAL  DE RIESGO QUIMICO',
 'NO USAR - PVE PARA LA PREVENCIÓN DEL RIESGO BIOLOGICO' : '139-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS POR RIESGO BIOLÓGICO EN SECTOR SALUD',
 'NO USAR - PVE PARA LA PREVENCIÓN DE LA HIPOACUSIA' : '142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA POR RUIDO',
 'NO USAR - PVE PARA LA PREVENCIÓN DE LA NEUMOCONIOSIS' : '143-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS SOBRE LA SALUD POR ASBESTO, SÍLICE Y/O CARBÓN',
 'NO USAR - RIESGO ELECTRICO' : '157-TRABAJO SEGURO CON RIESGO ELECTRICO',
 'NO USAR - PVE  PARA LA PREVENCIÓN DE DESORDENES MUSCULO ESQUELETICOS' : '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)',
 'NO USAR - GESTION ADMINISTRATIVA' : '181-GESTION ADMINISTRATIVA',
 'NO USAR - ESTRUCTURA Y GESTIÓN DEL RIESGO PSICOSOCIAL Y LA SALUD MENTAL' : '167-GESTIÓN PSICOSOCIAL Y PROMOCION DE LA SALUD MENTAL',
 'NO USAR - SISTEMA DE GESTION EN SST' : '170-SISTEMA DE GESTIÓN EN SST',
 'NO USAR - PVE PARA LA PREVENCION DEL RIESGO DE RADIACIONES IONIZANTES' : '172-PVE Y ACCIONES PARA PREVENCION DE LOS EFECTOS SOBRE LAS SALUD POR LAS  RADIACIONES IONIZANTES',
 'NO USAR - PVE PARA LA PREVENCION DE RIESGO QUIMICO' : '173-PVE Y ACCIONES PARA PREVENCION DE LOS EFECTOS SOBRE LA SALUD POR AGENTES DE RIESGO QUIMICO',
 'NO USAR - APOYO A LA PROMOCIÓN Y PREVENCIÓN' : '175-APOYO A LA PROMOCIÓN Y PREVENCIÓN',
 'NO USAR - ARIADNA' : '176-ARIADNA',
 'NO USAR - CONTRATISTAS' : '177-CONTRATISTAS',
 'NO USAR - CULTURA EN SEGURIDAD Y SALUD EN EL TRABAJO' : '178-CULTURA EN SEGURIDAD Y SALUD EN EL TRABAJO',
 'NO USAR - LABORATORIO DE HIGIENE' : '182-LABORATORIO DE HIGIENE',
 'NO USAR - LÍDERES EN SALUD Y SEGURIDAD EN EL TRABAJO' : '183-LÍDERES EN SALUD Y SEGURIDAD EN EL TRABAJO',
 'NO USAR - MERCADEO COMO APOYO A LA PROMOCIÓN Y PREVENCIÓN' : '185-MERCADEO COMO APOYO A LA PROMOCIÓN Y PREVENCIÓN',
 'NO USAR - PROGRAMA PARA LA PROMOCION DE LA SALUD CARDIOVASCULAR' : '186-PROGRAMA DE ESTILOS DE VIDA SALUDABLES Y PROMOCIÓN DE LA SALUD',
 'NO USAR - PVE PARA LA PREVENCIÓN DEL RIESGO BIOLOGICO - DECRETO 488' : '188-PVE PARA LA PREVENCIÓN DEL RIESGO BIOLOGICO - DECRETO 488',
 'NO USAR - REINCORPORACIÓN LABORAL' : '190-REINCORPORACIÓN LABORAL',
 'NO USAR - SEGURIDAD DE PROCESOS' : '191-SEGURIDAD DE PROCESOS',
 'NO USAR - SVE PARA RIESGO BIOLOGICO- COVID 19' : '192-SVE PARA RIESGO BIOLOGICO- COVID 19',
 'NO USAR - SOBREESFUERZOS (ATEL)' : '158-PVE Y ACCIONES PARA LA GESTIÓN DEL RIESGO BIOMECANICO',
 'NO USAR - ATENCIÓN PYMES PROTOCOLO 1' : '189-PYMES',
 'NO USAR - ATENCIÓN PYMES PROTOCOLO 2' : '189-PYMES',
 'NO USAR - ATENCIÓN PYMES PROTOCOLO 3' : '189-PYMES',
 'NO USAR - ATENCIÓN PYMES PROTOCOLO 4' : '189-PYMES',
 'NO USAR - ATENCIÓN PYMES PROTOCOLO 5 (SOLO BOGOTA)' : '189-PYMES',
 'NO USAR - PYMES' : '189-PYMES',
 '158-PVE Y ACCIONES PARA LA GESTIÓN DEL RIESGO BIOMECANICO' : '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)',
 '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA' : '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)',
 'NO USAR - 142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA P' : '142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA POR RUIDO',
 'NO USAR - 143-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS SOBRE LA SALUD POR ASBESTO,' : '143-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS SOBRE LA SALUD POR ASBESTO, SÍLICE Y/O CARBÓN',
 'NO USAR - 139-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS POR RIESGO BIOLÓGICO EN SEC' : '139-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS POR RIESGO BIOLÓGICO EN SECTOR SALUD',
 'NO USAR - 173-PVE Y ACCIONES PARA PREVENCION DE LOS EFECTOS SOBRE LA SALUD POR AGENTES DE ' : '173-PVE Y ACCIONES PARA PREVENCION DE LOS EFECTOS SOBRE LA SALUD POR AGENTES DE RIESGO QUIMICO',
 '158-PVE Y ACCIONES PARA LA GESTIÓN DEL RIESGO BIOMECANICO' : '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)', 
 '142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA P' : '142-PVE Y ACCIONES PARA LA PREVENCION DE LA HIPOACUSIA NEUROSENSORIAL INDUCIDA POR RUIDO',
 'NO USAR -  LABORATORIO DE HIGIENE' : '182-LABORATORIO DE HIGIENE'
})

c = file['DSPRODUCTO'].unique()
print('Productos:',c)

Productos: ['133-GESTIÓN INTEGRAL  DE RIESGO QUIMICO'
 '158-PVE Y ACCIONES PARA PREVENCIÓN DE LOS EFECTOS POR RIESGOS ERGONÓMICOS (CARGA FÍSICA, VIBRACIÓN Y OTROS)'
 'NO USAR - INTERVENCIÓN DEL  AT Y EL'
 '131-TRABAJO SEGURO EN RIESGO MECANICO Y ENERGIAS PELIGROSAS'
 '170-SISTEMA DE GESTIÓN EN SST'
 '139-PVE Y ACCIONES PARA LA PREVENCIÓN DE LOS EFECTOS POR RIESGO BIOLÓGICO EN SECTOR SALUD'
 '167-GESTIÓN PSICOSOCIAL Y PROMOCION DE LA SALUD MENTAL'
 'EDUCACIÓN PARA EL CUIDADO' '189-PYMES' '128-TRABAJO SEGURO EN ALTURA'
 '175-APOYO A LA PROMOCIÓN Y PREVENCIÓN'
 '127-GESTION DE LA MOVILIDAD SEGURA Y SOSTENIBLE'
 '130-SEGURIDAD EN TRABAJOS EN CALIENTE'
 '112-PLAN DE PREVENCION, PREPARACION Y RESPUESTA ANTE EMERGENCIAS'
 '171-PREVENCIÓN DE CAÍDAS A NIVEL'
 '197-PROGRAMA DE LA PROMOCION DE LA SANA CONVIVENCIA Y GESTION DE LAS VIOLENCIAS'
 '124-GESTION DE ORDEN Y ASEO' '129-TRABAJO SEGURO EN ESPACIOS CONFINADOS'
 '157-TRABAJO SEGURO CON RIESGO ELECTRICO'
 '142-PVE Y ACCIONES PARA LA PREVENCION

In [29]:
#se estandariza codigos de los productos con los productos nuevos
file.CDPRODUCTO = file.CDPRODUCTO.replace({12 : 112, 24 : 124, 27 : 127, 28 : 128, 29 : 129, 
                                                               30 : 130, 31 : 131, 33 : 133, 39 : 139, 42 : 142,
                                                               43 : 143, 57 : 157, 58 : 158, 61 : 181, 67 : 167, 
                                                               11 : 170, 72 : 172, 73 : 173, 21 : 175, 2 : 176, 
                                                               81 : 177, 60 : 178, 7 : 182, 23 : 183, 74 : 185, 
                                                               71 : 186, 84 : 188, 59 : 190, 70 : 191, 83 : 192, 68 : 158, 
                                                               75 : 189,76 : 189,77 : 189, 78 : 189, 79 : 189,
                                                               174 : 189,  38 : 170,  40 : 170

})
file['CDPRODUCTO'].unique()

array([133, 158,   4, 131, 170, 139, 167,  13, 189, 128, 175, 127, 130,
       112, 171, 197, 124, 129, 157, 142, 173, 196, 172, 182, 186, 176,
       143, 191, 192, 183, 178, 177, 194, 195, 190], dtype=int64)

In [30]:
#VERIFICACIÓN POR OC
#filtro_oc = file_total['NMCONSECUTIVO_ORDEN']== '3803066'
#oc = file_total[filtro_oc]
#oc

In [31]:
#Se quitan citas canceladas y programadas
#file = file [(file['DSESTADO_PROGRAMACION']== 'CITA EJECUTADA')]
#file.info()

In [32]:
#Llave por OC
#convertir en str
file['NMCONSECUTIVO_ORDEN'] = file['NMCONSECUTIVO_ORDEN'].apply(str)
file['NMCONSECUTIVO_PLAN'] = file['NMCONSECUTIVO_PLAN'].apply(str)
file['NMACTIVIDAD'] = file['NMACTIVIDAD'].apply(str)
file['NMVERSION'] = file['NMVERSION'].apply(str)
file['Llave_oc'] = file.NMCONSECUTIVO_ORDEN + file.NMCONSECUTIVO_PLAN + file.NMVERSION + file.NMACTIVIDAD 

In [33]:
#Sumar cantidad ejecutada por llave oc
file_cantidad = pd.DataFrame(file[['Llave_oc', 'NMCANTIDAD_EJECUTADA']])
file_cantidad = file_cantidad.groupby(['Llave_oc']).agg({'NMCANTIDAD_EJECUTADA':'sum'})
file_cantidad = pd.DataFrame(file_cantidad)
file_cantidad.columns=['Total_ejecutado']
file_cantidad.head()

,Total_ejecutado
Llave_oc,
531794959819841,8.0
531795059819842,2.0
5317998613797428,1.5
5318000613797429,1.5
5318003613797430,1.5


In [34]:
#Consolidar 
file_total = pd.merge(left= file, right=file_cantidad, on= 'Llave_oc', how= 'left' )
file_total.drop_duplicates(['Llave_oc'], keep = 'last', inplace = True)
#file_total.info()

In [35]:
#Limpieza de datos - datos nulos
na = file_total.isnull().any()
na_=file_total.isnull().sum()
print(na)
print('***************************')
print(na_)

NMCONSECUTIVO_ORDEN       False
NMCONSECUTIVO_PLAN        False
NMACTIVIDAD               False
NMVERSION                 False
DNI_DISTRIBUIDOR          False
NOMBRE_DISTRIBUIDOR       False
DNI_PRESTADOR             False
NOMBRE_PRESTADOR          False
TIPO_DE_ASESOR            False
NPOLIZA                   False
CDTAREA                   False
DSTAREA                   False
CDPRODUCTO                False
DSPRODUCTO                False
FEENTREGA_SERVICIO_INI    False
NMCANTIDAD_PEDIDA         False
NMCANTIDAD_EJECUTADA      False
DSESTADO_PROGRAMACION     False
CLASIFICACION             False
Llave_oc                  False
Total_ejecutado           False
dtype: bool
***************************
NMCONSECUTIVO_ORDEN       0
NMCONSECUTIVO_PLAN        0
NMACTIVIDAD               0
NMVERSION                 0
DNI_DISTRIBUIDOR          0
NOMBRE_DISTRIBUIDOR       0
DNI_PRESTADOR             0
NOMBRE_PRESTADOR          0
TIPO_DE_ASESOR            0
NPOLIZA                   0
CDTAREA 

In [36]:
#Se calcula pendiente por OC
file_total['Total_pendiente_OC'] = file_total['NMCANTIDAD_PEDIDA'] -  file_total['Total_ejecutado']
file_total.sort_values('Total_pendiente_OC', inplace = True)

In [37]:
# Convertir fecha en año
#convertir variable 'feentrega_tarea_fac'en fecha
file_total['FEENTREGA_SERVICIO_INI'] = pd.to_datetime(file_total['FEENTREGA_SERVICIO_INI'], format="%d/%m/%Y %H:%M:%S")
#Crear variable año
file_total['Año'] = file_total['FEENTREGA_SERVICIO_INI'].dt.year
#Crear variable mes
file_total['Mes']= file_total['FEENTREGA_SERVICIO_INI'].dt.month

In [38]:
#Verificar OC con mas ejecución que asignación
filtro_error =file_total['Total_pendiente_OC'] < 0
file_errores = file_total[filtro_error]
print('Número de OC con errores encontrados', file_errores['Total_pendiente_OC'].count())
#Corregir errores
file_total['Total_pendiente_OC'] = np.where(file_total['Total_pendiente_OC'] < 0, file_total['Total_ejecutado'],
                                         file_total['Total_pendiente_OC'])
#Verificar OC con mas ejecución que asignación
filtro_error =file_total['Total_pendiente_OC'] < 0
file_errores = file_total[filtro_error]
print('Número de OC con errores restantes', file_errores['Total_pendiente_OC'].count())

Número de OC con errores encontrados 59
Número de OC con errores restantes 0


In [39]:
file_prestador = pd.DataFrame(file_total[['DNI_DISTRIBUIDOR', 'NOMBRE_DISTRIBUIDOR', 'DNI_PRESTADOR','NOMBRE_PRESTADOR', 'FEENTREGA_SERVICIO_INI', 
                                          'NMCANTIDAD_PEDIDA', 'NMCANTIDAD_EJECUTADA','Total_ejecutado', 'Año', 'Mes', 'Total_pendiente_OC']])

In [40]:
#Llave prestador mes
##Convertir mes y año en str
file_prestador['Mes'] = file_prestador['Mes'].apply(str)
file_prestador['Año'] = file_prestador['Año'].apply(str)
#Llave prestador mes
file_prestador['Prestador_mes_año'] = file_prestador.DNI_DISTRIBUIDOR + file_prestador.DNI_PRESTADOR + file_prestador.Mes + file_prestador.Año
file_prestador['Llave_prestador'] = file_prestador.DNI_DISTRIBUIDOR + file_prestador.DNI_PRESTADOR
#Consolidar por prestador mes y año
file_mes = pd.DataFrame(file_prestador[['Prestador_mes_año', 'Total_pendiente_OC']])
#Sumer cantidad por tarea, prestador, mes
file_mes = file_mes.groupby(['Prestador_mes_año']).agg({'Total_pendiente_OC':'sum'})
file_mes = pd.DataFrame(file_mes)
file_mes.columns=['Pendiente_mes']
file_mes.tail()
#consolidar
file_prestador = pd.merge(left= file_prestador, right=file_mes, on= 'Prestador_mes_año', how= 'left' )
file_prestador.sort_values('Pendiente_mes', inplace = True)
#file_prestador.drop_duplicates(['Prestador_mes_año'], keep = 'last', inplace = True)
#file_prestador.tail()

In [41]:
#Llave prestador mes
##Convertir mes y año en str
file_prestador['Mes'] = file_prestador['Mes'].apply(str)
file_prestador['Año'] = file_prestador['Año'].apply(str)
#Llave prestador mes
file_prestador['Prestador_mes_año'] = file_prestador.DNI_DISTRIBUIDOR + file_prestador.DNI_PRESTADOR + file_prestador.Mes + file_prestador.Año
file_prestador['Llave_prestador'] = file_prestador.DNI_DISTRIBUIDOR + file_prestador.DNI_PRESTADOR
#Consolidar por prestador mes y año
file_mes = pd.DataFrame(file_prestador[['Prestador_mes_año', 'NMCANTIDAD_PEDIDA']])
#Sumer cantidad por tarea, prestador, mes
file_mes = file_mes.groupby(['Prestador_mes_año']).agg({'NMCANTIDAD_PEDIDA':'sum'})
file_mes = pd.DataFrame(file_mes)
file_mes.columns=['Asigando_mes']
file_mes.tail()
#consolidar
file_prestador = pd.merge(left= file_prestador, right=file_mes, on= 'Prestador_mes_año', how= 'left' )
file_prestador.sort_values('Asigando_mes', inplace = True)
file_prestador.drop_duplicates(['Prestador_mes_año'], keep = 'last', inplace = True)
file_prestador.tail()

,DNI_DISTRIBUIDOR,NOMBRE_DISTRIBUIDOR,DNI_PRESTADOR,NOMBRE_PRESTADOR,FEENTREGA_SERVICIO_INI,NMCANTIDAD_PEDIDA,NMCANTIDAD_EJECUTADA,Total_ejecutado,Año,Mes,Total_pendiente_OC,Prestador_mes_año,Llave_prestador,Pendiente_mes,Asigando_mes
314687,C93379730,MENDEZ CALDERON MARIO FERNANDO,C93379730,MENDEZ CALDERON MARIO FERNANDO,2022-03-03,2.0,2.0,2.0,2022,3,0.0,C93379730C9337973032022,C93379730C93379730,18.0,1890.0
306376,N830052848,PROTEGER PROFESIONALES EN PREVENCION DE RIESGO...,C52521366,LUGO JIMENEZ NASLY JOHANNA,2022-10-06,2.0,2.0,2.0,2022,10,0.0,N830052848C52521366102022,N830052848C52521366,16.0,2022.5
551618,N860012336,INSTITUTO COLOMBIANO DE NORMAS TECNICAS ICONTEC,N860012336,INSTITUTO COLOMBIANO DE NORMAS TECNICAS ICONTEC,2022-07-12,1.0,1.0,1.0,2022,7,0.0,N860012336N86001233672022,N860012336N860012336,1050.0,2100.0
321588,N830052848,PROTEGER PROFESIONALES EN PREVENCION DE RIESGO...,C52521366,LUGO JIMENEZ NASLY JOHANNA,2022-04-29,2.0,2.0,2.0,2022,4,0.0,N830052848C5252136642022,N830052848C52521366,20.0,3324.0
179245,N901007540,WCT WORLD CLASS TRANSPORT SAS,C79666276,FORIGUA ALFONSO JULIO HERNANDO,2022-08-22,2.0,2.0,2.0,2022,8,0.0,N901007540C7966627682022,N901007540C79666276,2.0,3736.0


In [42]:
##Verificación de valores
a= file_total['Total_pendiente_OC'].sum()
b =file_prestador['Pendiente_mes'].sum()
print('Total pendiente', a)
print ('********')
print('Pendiente mes', b)

Total pendiente 1556312.03
********
Pendiente mes 1556312.03


In [43]:
##Verificación de valores
a= file_total['NMCANTIDAD_PEDIDA'].sum()
b =file_prestador['Asigando_mes'].sum()
print('Total asignado', a)
print ('********')
print('Asignado mes', b)

Total asignado 3920927.3
********
Asignado mes 3920927.3


In [44]:
# Filtrar por años anteriores
file_prestador['Año'] = file_prestador['Año'].apply(int)
file_prestador['Mes'] = file_prestador['Mes'].apply(int)
filtro_pasado =file_prestador['Año'] < datetime.datetime.now().year
file_pasado = file_prestador[filtro_pasado]
print('Años', file_pasado['Año'].unique())

Años [2022 2020 2021]


In [45]:
#Pendiente de ejecutar
file_cantidad = pd.DataFrame(file_pasado[['Llave_prestador', 'Pendiente_mes']])
file_cantidad = file_cantidad.groupby(['Llave_prestador']).agg({'Pendiente_mes':'sum'})
file_cantidad = pd.DataFrame(file_cantidad)
file_cantidad.columns=['Pendiente_pasado']

file_cantidad.tail()

,Pendiente_pasado
Llave_prestador,
N901593963C1030521121,8.0
N901593963C52047164,0.0
N901593963C52047329,2.0
N901595786C1030558879,26.0
N901595786C19442917,1.5


In [46]:
# FILTRAR AÑO y MES acutal
# Filtrar por año actual
filtro_actual=file_prestador['Año'] == datetime.datetime.now().year
file_actual = file_prestador[filtro_actual]
print('Año', file_actual['Año'].unique())
#Mes actual
#filtrar mes actual
filtro_mes = file_actual['Mes'] < datetime.datetime.now().month
file_actual = file_actual[filtro_mes]
print('Meses', file_actual['Mes'].unique())

Año [2023]
Meses [4 2 3 1]


In [47]:
#Sumar cantidad pendiente de ejecutar por llave oc
file_cantidad2 = pd.DataFrame(file_actual[['Llave_prestador', 'Pendiente_mes']])
file_cantidad2 = file_cantidad2.groupby(['Llave_prestador']).agg({'Pendiente_mes':'sum'})
file_cantidad2 = pd.DataFrame(file_cantidad2)
file_cantidad2.columns=['Pasado_2023']
file_cantidad2.head()

,Pasado_2023
Llave_prestador,
C10017893C10017893,16.0
C1014231610C1014231610,10.0
C1015992923C1015992923,6.0
C1016082052C1016082052,8.0
C1018446175C1018446175,8.0


In [48]:
# FILTRAR AÑO 2023 y mes
# Filtrar por año actual
filtro_actual=file_prestador['Año'] == datetime.datetime.now().year
file_actual = file_prestador[filtro_actual]
print('Año', file_actual['Año'].unique())
file_actual = pd.DataFrame(file_actual[['Llave_prestador', 'Mes', 'Asigando_mes']])
#Restablecer el indice
file_actual.reset_index(drop=True, inplace=True)
#filtrar mes de 2023
filtro_mes = file_actual['Mes'] >= datetime.datetime.now().month
file_actual = file_actual[filtro_mes]
print('Meses', file_actual['Mes'].unique())

Año [2023]
Meses [ 5 12  6 10  8 11  7  9]


In [49]:
#verificar
filtro_prestador = file_prestador['Llave_prestador']== 'C46673663C46673663'
df = file_prestador[filtro_prestador]
#df.head(5)

In [50]:
#Se crea una tabla de cantidad pendiente mes por llave prestador
file_actual = (file_actual.groupby(['Llave_prestador', 'Mes'], as_index = False).sum().pivot('Llave_prestador', 'Mes').fillna(0))
file_actual

Asigando_mes                                        
Mes                              5     6     7     8     9     10   11   12
Llave_prestador                                                            
C10017893C10017893              6.0  24.0  30.0  19.0  12.0  14.0  0.0  0.0
C1014231610C1014231610          2.0   2.0   0.0   2.0   4.0   4.0  0.0  2.0
C1015992923C1015992923          2.0   0.0   2.0   2.0   0.0   4.0  0.0  0.0
C1016082052C1016082052          0.0   2.0   0.0   2.0   2.0   0.0  4.0  0.0
C1018446175C1018446175          2.0   0.0   2.0   2.0   2.0   2.0  2.0  2.0
...                             ...   ...   ...   ...   ...   ...  ...  ...
N901650603N901650603            1.0   1.0   1.0   0.0   1.0   0.0  1.0  0.0
N901660775C1035428178           0.0   4.0   4.0  20.0  10.0   0.0  0.0  0.0
N901660775C1039467964           0.0  10.0   2.0   5.0   3.0   0.0  0.0  0.0
N901660775C1094247091           8.0   0.0   0.0   0.0   0.0   0.0  0.0  0.0
N901674319N901674319            1.0   0.0   1.0   0.0   1.0   0.0  0.0  0.0

[2755 rows x 8 columns]

In [51]:
#Se unen los dataframes
file_final = pd.DataFrame(file_prestador[['DNI_DISTRIBUIDOR', 'NOMBRE_DISTRIBUIDOR', 'DNI_PRESTADOR','NOMBRE_PRESTADOR', 'Llave_prestador']])
file_final.drop_duplicates(['Llave_prestador'], keep = 'last', inplace = True)
file_final = pd.merge(left= file_final, right=file_cantidad, on= 'Llave_prestador', how= 'left' )
file_final = pd.merge(left= file_final, right=file_cantidad2, on= 'Llave_prestador', how= 'left' )
file_final = pd.merge(left= file_final, right=file_actual, on= 'Llave_prestador', how= 'left' )
#file_final.head(5)

In [52]:
## Renombrar columnas
#Sacamos los nombres de las columnas de df
column_names = file_actual.columns.tolist()
#Creamos un diccionario
new_column_names = {col: f'Asignado_{i +datetime.datetime.now().month}' for i, col in enumerate(column_names)}
# Renombrar las columnas utilizando el diccionario
file_final = file_final.rename(columns=new_column_names)
file_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3875 entries, 0 to 3874
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DNI_DISTRIBUIDOR     3875 non-null   object 
 1   NOMBRE_DISTRIBUIDOR  3875 non-null   object 
 2   DNI_PRESTADOR        3875 non-null   object 
 3   NOMBRE_PRESTADOR     3875 non-null   object 
 4   Llave_prestador      3875 non-null   object 
 5   Pendiente_pasado     3294 non-null   float64
 6   Pasado_2023          2913 non-null   float64
 7   Asignado_5           2755 non-null   float64
 8   Asignado_6           2755 non-null   float64
 9   Asignado_7           2755 non-null   float64
 10  Asignado_8           2755 non-null   float64
 11  Asignado_9           2755 non-null   float64
 12  Asignado_10          2755 non-null   float64
 13  Asignado_11          2755 non-null   float64
 14  Asignado_12          2755 non-null   float64
dtypes: float64(10), object(5)
memory usage

In [53]:
#cambiar datos nulos por cero
file_final.fillna(0, inplace = True)

In [54]:
#Verificación - datos nulos
na = file_final.isnull().any()
na_=file_final.isnull().sum()
print(na)
print('***************************')
print(na_)

DNI_DISTRIBUIDOR       False
NOMBRE_DISTRIBUIDOR    False
DNI_PRESTADOR          False
NOMBRE_PRESTADOR       False
Llave_prestador        False
Pendiente_pasado       False
Pasado_2023            False
Asignado_5             False
Asignado_6             False
Asignado_7             False
Asignado_8             False
Asignado_9             False
Asignado_10            False
Asignado_11            False
Asignado_12            False
dtype: bool
***************************
DNI_DISTRIBUIDOR       0
NOMBRE_DISTRIBUIDOR    0
DNI_PRESTADOR          0
NOMBRE_PRESTADOR       0
Llave_prestador        0
Pendiente_pasado       0
Pasado_2023            0
Asignado_5             0
Asignado_6             0
Asignado_7             0
Asignado_8             0
Asignado_9             0
Asignado_10            0
Asignado_11            0
Asignado_12            0
dtype: int64


In [55]:
#Se totaliza lo pendiente pasado
file_final['Total_pasado'] = file_final['Pendiente_pasado'] +  file_final['Pasado_2023']
file_final.drop_duplicates(subset=['Llave_prestador'], keep=False, inplace=True)
p =file_final['Total_pasado'].sum()
print('Total_pasado', p, 'horas')

Total_pasado 528545.13 horas


In [56]:
file_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3875 entries, 0 to 3874
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DNI_DISTRIBUIDOR     3875 non-null   object 
 1   NOMBRE_DISTRIBUIDOR  3875 non-null   object 
 2   DNI_PRESTADOR        3875 non-null   object 
 3   NOMBRE_PRESTADOR     3875 non-null   object 
 4   Llave_prestador      3875 non-null   object 
 5   Pendiente_pasado     3875 non-null   float64
 6   Pasado_2023          3875 non-null   float64
 7   Asignado_5           3875 non-null   float64
 8   Asignado_6           3875 non-null   float64
 9   Asignado_7           3875 non-null   float64
 10  Asignado_8           3875 non-null   float64
 11  Asignado_9           3875 non-null   float64
 12  Asignado_10          3875 non-null   float64
 13  Asignado_11          3875 non-null   float64
 14  Asignado_12          3875 non-null   float64
 15  Total_pasado         3875 non-null   f

In [57]:
#exportar
#file_total.to_csv('facturado_depurado.csv', index = False)
#df = df.drop(labels= ['FEENTREGA_SERVICIO_INI'], axis = 1)
#df.to_excel('verificar.xlsx', index = False)
file_final.to_excel('Asignado_depurado.xlsx', index = False)
#df2.to_excel('Facturado_sin_depurar.xlsx', index= False)

## Analísis Consolidado <a class= "anchor" id= "fourth-bullet"></a>

[Tabla de contenido](#table-bullet)

In [58]:
#Se organiza el DF para el merge
capacidad = red_e_depurada.drop(labels= ['Llave', 'DNI_PRESTADOR_y', 'DNI_DISTRIBUIDOR_y', 'DSCLASIFICACION' ], axis = 1)
ocupacion = file_final.drop(labels=['DNI_DISTRIBUIDOR', 'NOMBRE_DISTRIBUIDOR', 'DNI_PRESTADOR', 'NOMBRE_PRESTADOR'], axis = 1)

In [59]:
#Renombrar columnas de capacidad
capacidad.rename(columns={'CAPACIDAD1': 1, 'CAPACIDAD2': 2, 'CAPACIDAD3': 3, 'CAPACIDAD4': 4,
                         'CAPACIDAD5': 5, 'CAPACIDAD6': 6, 'CAPACIDAD7': 7, 'CAPACIDAD8': 8,
                         'CAPACIDAD9': 9, 'CAPACIDAD10': 10, 'CAPACIDAD11': 11, 'CAPACIDAD12': 12}, inplace=True)
#Borrar capaciad de meses pasados
#Creamos una lista de las columnas a eliminar
columnas_eliminar = [mes for mes in range (1, 13) if mes < datetime.datetime.now().month]
#Eliminar columnas de meses pasados
capacidad = capacidad.drop(labels=columnas_eliminar, axis=1)

In [60]:
#Unir los dos dataframes
file_ocupacion = pd.merge(left= capacidad, right=ocupacion, on= 'Llave_prestador', how= 'left')

In [61]:
#cambiar datos nulos por cero
file_ocupacion.fillna(0, inplace = True)

In [62]:
#Limpieza de datos - datos nulos
na = file_ocupacion.isnull().any()
na_=file_ocupacion.isnull().sum()
print(na)
print('***************************')
print(na_)

DNI_PRESTADOR_x        False
NOMBRE_PRESTADOR       False
PERFIL_TARIFA          False
DNI_DISTRIBUIDOR_x     False
NOMBRE_DISTRIBUIDOR    False
CDPRODUCTO             False
DSPRODUCTO             False
TIPO_DE_RED            False
CDMUNICIPIO            False
DSMUNICIPIO            False
FUNCION_PRESTADOR      False
SNVALIDADO             False
CAPACIDAD              False
DSTIPO_PERFIL          False
Llave_prestador        False
5                      False
6                      False
7                      False
8                      False
9                      False
10                     False
11                     False
12                     False
Capacidad_total        False
Pendiente_pasado       False
Pasado_2023            False
Asignado_5             False
Asignado_6             False
Asignado_7             False
Asignado_8             False
Asignado_9             False
Asignado_10            False
Asignado_11            False
Asignado_12            False
Total_pasado  

In [63]:
#Restar la capacidad del pasado a la capacidad del mes actual
mes_actual = 'Asignado_' + str(datetime.datetime.now().month)
#mes_actual
file_ocupacion['Disponibilidad_actual'] = file_ocupacion[datetime.datetime.now().month] - (file_ocupacion['Total_pasado'] + file_ocupacion[mes_actual])

In [64]:
#Columnas capacidad futura
columnas_capacidad = [mes for mes in range (1, 13) if mes > datetime.datetime.now().month]
#Columnas pendiete futuro
columnas_asignado = ['Asignado_{}'.format(mes) for(mes) in range (datetime.datetime.now().month+1,13)]
print('Columnas capaciadad: ', columnas_capacidad)
print('Columnas pendiente: ', columnas_asignado)

Columnas capaciadad:  [6, 7, 8, 9, 10, 11, 12]
Columnas pendiente:  ['Asignado_6', 'Asignado_7', 'Asignado_8', 'Asignado_9', 'Asignado_10', 'Asignado_11', 'Asignado_12']


In [65]:
#Restar capaciad y asignado
# Crear las nuevas columnas con el resultado de la resta
for idx, col_asignado in enumerate(columnas_asignado):
    col_capacidad = columnas_capacidad[idx]
    col_resultado = 'Disponibilidad_{}'.format(col_asignado.split('_')[1])
    file_ocupacion[col_resultado] = file_ocupacion[col_capacidad] - file_ocupacion[col_asignado]

In [66]:
#exportar
#total_ac.to_csv('Red_externa_asesoría_y_capacitación.csv', index = False)
#print('El resultado se ha exportado en un archivo de CSV')
file_ocupacion.to_excel(r'C:\Users\dianvaja\OneDrive - Seguros Suramericana, S.A\Documentos\PLANIFICACIÓN\Conciliación oferta demanda\Disponibilidad de la red.xlsx', index = False)
print('El resultado se ha exportado en un archivo de Excel')

El resultado se ha exportado en un archivo de Excel


In [67]:
fin = time.time()
t= (fin-inicio)/60
print("Tiempo de ejecución:", t, "minutos")

Tiempo de ejecución: 1.4402974247932434 minutos
